In [55]:
from pathlib import Path
cwd = str(Path.cwd().parents[0])
import sys
sys.path.append(cwd)
from engine.sql.almaden import Sql
import selenium
from selenium import webdriver
from selenium.webdriver import ActionChains
from bs4 import BeautifulSoup
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import Select
from selenium.webdriver.support.ui import WebDriverWait
import time
import requests
import datetime
import pandas as pd
import pickle
from contextlib import suppress
import re
from html import unescape

In [56]:
# 결과 저장 경로
save_path = ""
task_id = "156242"
# 카페 정보
cafe = {'name':"문명 메트로폴리스"                          # 카페 이름
       ,'page_link':"https://cafe.naver.com/civilization4"}                     # 주소
cafe.update({"keywords" : ["레볼루션"]})         # 검색 키워드

In [57]:
### 카페 주소 입력
chrome_options = webdriver.ChromeOptions()
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=chrome_options)
db = Sql("datacast2")



====== WebDriver manager ======
Current google-chrome version is 98.0.4758
Get LATEST chromedriver version for 98.0.4758 google-chrome
Trying to download new driver from https://chromedriver.storage.googleapis.com/98.0.4758.102/chromedriver_win32.zip
Driver has been saved in cache [C:\Users\wodud\.wdm\drivers\chromedriver\win32\98.0.4758.102]


In [58]:
driver.get(cafe.get("page_link"))

In [ ]:
###카페 전체게시글 가져오기
WebDriverWait(driver,3).until(EC.presence_of_element_located((By.ID,'menuLink0')))
driver.find_element_by_xpath('//*[@id="menuLink0"]').click()
driver.implicitly_wait(0.5)
driver.switch_to.frame('cafe_main')

In [54]:
for keyword in cafe.get("keywords"):
    ### 카페 주소 입력


    ### 키워드 검색
    # search_input = driver.find_element_by_css_selector("#topLayerQueryInput")
    search_input = driver.find_element(By.CSS_SELECTOR, "#topLayerQueryInput")
    search_input.send_keys("레볼루션")
    try :
        driver.find_element(By.XPATH,'//*[@id="cafe-search"]/form/button').click()   # 왼쪽에 검색창
        # driver.find_element_by_xpath('//*[@id="cafe-search"]/form/button').click()   # 왼쪽에 검색창
    except:
        driver.find_element(By.XPATH,'//*[@id="info-search"]/form/button').click()   # 오른쪽에 검색창
        # driver.find_element_by_xpath('//*[@id="info-search"]/form/button').click()   # 오른쪽에 검색창

    driver.implicitly_wait(0.5)
    driver.switch_to.frame('cafe_main')

    ### 키워드 수집 정보
    num_per_page = 15          # 페이지당 게시글 갯수(default: 15개)

    address_list=[]
    page = 1

    l=True
    while l:
        time.sleep(1)
        ### 현재 페이지의 html 불러오기
        r = driver.page_source
        page_html = BeautifulSoup(r, "html.parser")
        try:
            content = page_html.find("div", class_="article-board result-board m-tcol-c").find('tbody')
        except:
            content = page_html.find_all("div", class_="article-board m-tcol-c")[1].find('tbody')
        body = content.find_all("tr")

        ### 게시글 정보 저장하기
        for x in body:
            temp_dict={}
            if x.find("div", class_="board-number") is not None:
                temp_dict['link'] = "https://cafe.naver.com" + x.find('a').get('href')
                temp_dict['name'] = x.find("td", class_="td_name").find('a',class_='m-tcol-c').text.strip()
                temp_dict['date'] = x.find("td", class_="td_date").text.strip().replace(".","-")[0:-1]
                temp_dict['view'] = x.find("td", class_="td_view").text.strip().replace(",","")
                if "만" in temp_dict['view']:
                    temp_dict['view'] = temp_dict['view'].replace("만","")
                    temp_dict['view'] = temp_dict['view'].replace(".","")
                    temp_dict['view'] = int(temp_dict['view'])*10000
                else :
                    temp_dict['view'] = int(temp_dict['view'])
                print(task_id, temp_dict['name'], int(temp_dict['view']),temp_dict['date'], temp_dict['link'] )
                # db.insert("crawl_contents",
                #       task_id= task_id,
                #       author = temp_dict['name'],
                #       n_view = int(temp_dict['view']),
                #       post_date = temp_dict['date'],
                #       url=temp_dict['link'],
                #       crawl_status = 'F')
        print("(현재시각) "+str(datetime.datetime.now())+": "+ str(page) +"page done")

        ### 다음 페이지로 넘어가기
        page+=1
        driver.implicitly_wait(1)
        try:
            if page<=10:   # 1~10 : 페이지 번호 그대로
                page_xpath = str(page)
                driver.find_element(By.XPATH,'//*[@id="main-area"]/div[7]/a[' + page_xpath + ']').click()
                # driver.find_element_by_xpath('//*[@id="main-area"]/div[7]/a[' + page_xpath + ']').click()
            elif page == 11:   # 11 : 다음 버튼
                driver.find_element(By.XPATH,'//*[@id="main-area"]/div[7]/a[11]/span').click()
                # driver.find_element_by_xpath('//*[@id="main-area"]/div[7]/a[11]/span').click()
            elif page>11 and page%10!=1:   # 12~ : 페이지 번호 마지막 자리 + 1
                page_xpath = str(page-((page-1)//10)*10+1)
                driver.find_element(By.XPATH,'//*[@id="main-area"]/div[7]/a[' + page_xpath + ']').click()
                # driver.find_element_by_xpath('//*[@id="main-area"]/div[7]/a[' + page_xpath + ']').click()
            elif page%10 == 1:   # 21,31.. : 다음 버튼
                driver.find_element(By.XPATH, '//*[@id="main-area"]/div[7]/a[12]/span').click()
                # driver.find_element_by_xpath('//*[@id="main-area"]/div[7]/a[12]/span').click()
        except Exception as e:
                print(e)
                l=False


28
62
80
26
55
80
180
196
52
41
65
222
38
841
72
(현재시각) 2022-02-18 18:51:17.321054: 1page done
58
53
187
119
130
63
100
62
73
86
153
60
79
83
136
(현재시각) 2022-02-18 18:51:18.852787: 2page done
105
336
79
231
346
426
133
331
363
118
1454
268
162
144
48
(현재시각) 2022-02-18 18:51:20.442036: 3page done
365
38
89
68
68
208
55
67
56
63
65
393
472
419
90
(현재시각) 2022-02-18 18:51:21.929771: 4page done
75
91
266
95
243
118
131
55
128
102
100
53
331
118
116
(현재시각) 2022-02-18 18:51:23.443647: 5page done
84
102
99
405
66
124
148
138
121
127
176
109
119
21
46
(현재시각) 2022-02-18 18:51:24.921827: 6page done
89
95
75
208
111
39
350
209
520
508
459
101
363
155
169
(현재시각) 2022-02-18 18:51:26.409676: 7page done
208
65
60
133
97
115
75
61
211
138
581
93
96
119
401
(현재시각) 2022-02-18 18:51:27.873766: 8page done
155
78
190
67
94
63
167
152
147
434
89
213
60
61
87
(현재시각) 2022-02-18 18:51:29.361024: 9page done
130
106
523
194
95
160
53
143
133
163
151
59
39
125
121
(현재시각) 2022-02-18 18:51:30.831091: 10page done
150

WebDriverException: Message: chrome not reachable
  (Session info: chrome=98.0.4758.102)
Stacktrace:
Backtrace:
	Ordinal0 [0x00A369A3+2582947]
	Ordinal0 [0x009CA6D1+2139857]
	Ordinal0 [0x008C3960+1063264]
	Ordinal0 [0x008B8092+1015954]
	Ordinal0 [0x008B88C8+1018056]
	Ordinal0 [0x008BA102+1024258]
	Ordinal0 [0x008B3DE9+998889]
	Ordinal0 [0x008C4DD0+1068496]
	Ordinal0 [0x00917B12+1407762]
	Ordinal0 [0x00908366+1344358]
	Ordinal0 [0x008E5176+1200502]
	Ordinal0 [0x008E6066+1204326]
	GetHandleVerifier [0x00BDBE02+1675858]
	GetHandleVerifier [0x00C9036C+2414524]
	GetHandleVerifier [0x00ACBB01+560977]
	GetHandleVerifier [0x00ACA8D3+556323]
	Ordinal0 [0x009D020E+2163214]
	Ordinal0 [0x009D5078+2183288]
	Ordinal0 [0x009D51C0+2183616]
	Ordinal0 [0x009DEE1C+2223644]
	BaseThreadInitThunk [0x761BFA29+25]
	RtlGetAppContainerNamedObjectPath [0x77EB7A9E+286]
	RtlGetAppContainerNamedObjectPath [0x77EB7A6E+238]


In [ ]:
from pathlib import Path
cwd = str(Path.cwd().parents[0])
import sys
sys.path.append(cwd)
from engine.sql.almaden import Sql
import selenium
from selenium import webdriver
from selenium.webdriver import ActionChains
from bs4 import BeautifulSoup
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By

from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import Select
from selenium.webdriver.support.ui import WebDriverWait
import time
import requests
import datetime
import pandas as pd
import pickle
from contextlib import suppress
import re
from html import unescape


# 결과 저장 경로
save_path = ""
task_id = "28320" 
# 카페 정보
cafe = {'name':"세븐나이츠"                          # 카페 이름
       ,'page_link':"https://cafe.naver.com/sevenknights"}                     # 주소
cafe.update({"keywords" : ["레볼루션"]})         # 검색 키워드

db = Sql("datacast2")
driver = webdriver.Chrome(executable_path=f'{cwd}/chromedriver.exe')
content_rows = db.select("crawl_contents","*",f"task_id={task_id}")
for keyword in cafe.get("keywords"):

    i=0
    contents_list = []   # 내용
    reply_list = []      # 댓글
    error_list = []      # 에러난 게시글

    while True:

        ### 수집 링크로 이동
        url = content_rows[i]['url']
        contents_id = content_rows[i]['contents_id']
        driver.get(url)
        time.sleep(1)
        try:
            driver.switch_to.frame('cafe_main')
            time.sleep(1)
            r = driver.page_source
            page_soup = BeautifulSoup(r, "html.parser")
            content = page_soup.find('div', class_='ArticleContentBox')  

            ### 게시글 수집
            temp_dict={}
            temp_dict['title'] = ""
            with suppress(AttributeError):   # 제목 없는 게시글
                temp_dict['title'] = content.find('h3',class_='title_text').text.strip()
            temp_dict['text'] = content.find("div", class_="article_viewer").text.strip()
            temp_dict['text'] = temp_dict['text'].replace("\n","")
            temp_dict['text'] = temp_dict['text'].replace("\u200b","")
            temp_dict['author'] = content.find('div',class_='profile_info').find('a',class_='nickname').text.strip()
            db.update_multi("crawl_contents",temp_dict,{"contents_id":contents_id})
            print(temp_dict)
            contents_list.append(temp_dict)
            
            ### 댓글 수집
            if content.find("div", class_="ReplyBox") is not None:   # 댓글 기능이 아예 없음  
                comment_num = content.find("div", class_="ReplyBox").find("a",class_="button_comment").find("strong").text
                if comment_num!='0':   # 댓글이 없음
                    comment = content.find("div", class_="CommentBox").find("ul",class_="comment_list").select("li")
                    
                    ### 댓글 구분
                    com_n=0    # 댓글
                    com_nn=0   # 대댓글
                    
                    for n in range(len(comment)):

                        if comment[n].get('class')==['CommentItem']:    # 댓글
                            com_n+=1; com_nn=0;
                            com_thread = str(com_n)+"-"+str(com_nn)
                            com_nn=1
                        elif comment[n].get('class')==['CommentItem', 'CommentItem--reply']:    # 대댓글
                            com_thread = str(com_n)+"-"+str(com_nn)
                            com_nn+=1

                        ### 댓글 내용 수집    
                        if comment[n].text.strip() != '삭제된 댓글입니다.':
                            com_nick = comment[n].find("a",class_="comment_nickname").text.strip()
                            com_date = comment[n].find("span",class_="comment_info_date").text.strip()
                            com_reply = comment[n].find("div",class_="comment_text_box").text.strip()
                            com_reply = com_reply.replace("\n","").replace("\u200b","")
                            db.insert("crawl_contents",
                                      author = com_nick,
                                      task_id = task_id,
                                      url=url,
                                      text=com_reply,
                                      is_reply=1)
            i+=1

        except:
            i+=1
            ### 게시글을 볼 등급이 안됨
            if page_soup.find('strong', class_='emph') is not None:
                error_list.append({"error" :  page_soup.find('strong', class_='emph').text+"등급 필요"
                                   , "url" : url})
                pass
            ### 에러 따로 확인
            else:
                error_list.append({"error" : "에러 확인 필요"
                                   , "url" : url})
                pass

# 크롬 종료 
driver.quit()

In [ ]:
driver.quit()